In [ ]:
#! If you want to force use of CPU (before import of tensorflow)
import os
# os.environ['CUDA_VISIBLE_DEVICES']='-1'
# os.environ['OMP_NUM_THREADS'] = '4'
# os.environ['export OPENBLAS_NUM_THREADS']='4'

#! import as tf
import tensorflow as tf

#! If you want to limit number of threads, the mechine has 32 virtual cores (16 physical ones)
print('thread inter = ', tf.config.threading.get_inter_op_parallelism_threads() )
print('thread intra = ', tf.config.threading.get_intra_op_parallelism_threads() )
 
tf.config.threading.set_inter_op_parallelism_threads(4)
tf.config.threading.set_intra_op_parallelism_threads(4)

#! If you want to limit the GPU memory usage, the machine has ~24GB. For most applications a few GBs is enough.
#! If you max out what you specify progressively up the threshhold.
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.set_logical_device_configuration(
    physical_devices[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=20_000)])

  logical_devices = tf.config.list_logical_devices('GPU')

except:
  # Invalid device or cannot modify logical devices once initialized.
  pass

#! Will print out what devices you are using
tf.config.list_physical_devices()



import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

from mlkernels.tensorflow import Matern32, EQ, Matern12, Matern52


from LFM.NeuralNet import NeuralN
from LFM.utilities import pshape
from LFM.utilities import reparameterize, log_normal_pdf, expHighLowFunc_01, exp_start_end_x_func
from LFM.GICNet    import FFLayer, KIPLayer, GICNet

pi  = tf.constant(np.pi, tf.float32)
tfd = tfp.distributions
tfb = tfp.bijectors


dirname='data-lfmDiag-nslid-smallColLonTrain/'
os.makedirs(dirname, exist_ok=True)



In [ ]:
dimU = 3
dimZ = 2
dimW = 1
dimX = 3
                         #  x         y         t
domain     = tf.constant( [[0., 1.], [0., 1.], [0., 1.]] )

nxSamples  = 512
# 9^3

# currently in dataset                rho,  mu                                  rho, mu  
zDist = tfd.Uniform(low=tf.constant([0.8, 0.1], shape=[dimZ]), high=tf.constant([1., 1.], shape=[dimZ]))
wDist = tfd.Uniform(low=tf.constant([1.], shape=[dimW]), high=tf.constant([1.], shape=[dimW]))

e_bound = 0.

def raveled3DDomainGrid(resolution):
    x, y, t       = tf.linspace(domain[0][0], domain[0][1], resolution[0]), tf.linspace(domain[1][0], domain[1][1], resolution[1]), \
                    tf.linspace(domain[2][0], domain[2][1], resolution[2])
    X, Y, T       = np.meshgrid(x, y, t, indexing='ij')
    Xr, Yr, Tr    = tf.reshape(tf.constant(X, dtype=tf.float32), [-1,1]), tf.reshape(tf.constant(Y, dtype=tf.float32), [-1,1]),\
                    tf.reshape(tf.constant(T, dtype=tf.float32), [-1,1])
    XYTr          = tf.concat([Xr, Yr, Tr], 1)
    return XYTr

def sampleDomain(n):
    xrand = tf.random.uniform(minval=domain[0][0]+e_bound, maxval=domain[0][1]-e_bound, shape=[n, 1])
    yrand = tf.random.uniform(minval=domain[1][0]+e_bound, maxval=domain[1][1]-e_bound, shape=[n, 1])
    trand = tf.random.uniform(minval=domain[2][0]+e_bound, maxval=domain[2][1], shape=[n, 1])
    return tf.concat([xrand, yrand, trand], 1)

print('samples.shape', sampleDomain(10))

b      = zDist.sample()
xtvect = sampleDomain(nxSamples)
    
    

In [ ]:
# Define alpha network

dim_DeepO_out  = dimU * 2

alphaNet  = NeuralN('alpha', np.array([ 1, 1, dimZ]) )

numFreq = 100
nhAlpha = 200

activ_alpha = 'swish'

alphaNet.NN = tf.keras.Sequential([ 
                                tf.keras.layers.Dense( nhAlpha, input_dim=( numFreq * 2 + dimZ + dimW ), activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( nhAlpha, activation=activ_alpha),
                                tf.keras.layers.Dense( dim_DeepO_out ) ])

freqsX = tf.linspace(0., 1., numFreq)

freqs_x  = expHighLowFunc_01(1., 0.01, freqsX)[None,:]
freqs_y  = expHighLowFunc_01(1., 0.01, freqsX)[None,:]
freqs_t  = expHighLowFunc_01(1., 0.01, freqsX)[None,:]
freqs    = tf.concat([freqs_x, freqs_y, freqs_t], 1)
#now they are trainable
alphaNet.fLayer = [ FFLayer( freqs, trainable=True ) ]

alphaNet.lower_alpha = tf.constant(1e-4)
alphaNet.upper_w     = tf.constant(1.)

def alphaMap(self, z, w, x):

        z_tile  = tf.tile(tf.reshape(z, [1,-1]), [x.shape[0], 1])
        w_tile  = tf.tile(tf.reshape(w, [1,-1]), [x.shape[0], 1])
        print('x.shape', x.shape)

        xFreq   = self.fLayer[0]( x )
        x_input = tf.concat([xFreq, z_tile, w_tile], 1)

        print('x_input.shape', x_input.shape)
        mapped  = self.NN( x_input )
        
        mean, logvar = mapped[:,:dimU], mapped[:,dimU:2*dimU]
        
        tf.debugging.assert_all_finite(mean, 'mean not finite')
        tf.debugging.assert_all_finite(logvar, 'logvar not finite')

        logvar = self.dBoundwalpha_log( logvar )

        print('alpha mean.shape = ',   mean.shape)
        print('alpha logvar.shape = ', logvar.shape)

        return mean, logvar

from types import MethodType
alphaNet.call = MethodType(alphaMap, alphaNet)
alphaNet(tf.constant(1., shape=[dimZ]), tf.constant(1., shape=[dimW]), sampleDomain(30))
alphaNet.compile()
alphaNet.NN.summary()
alphaNet.summary()

# alphaNet.save_weights(dirname + '/alphaModel/alphaModel-lfmDiag')
# alphaNet.load_weights(dirname + '/alphaModel/alphaModel-lfmDiag')

# del alphaNet


In [ ]:
# Define beta network


dim_trunc_in_beta   = dimW
dim_trunc_out_beta  = 200 

dim_branch_in_y_beta = 3
dim_branch_P_beta    = 30
dv_beta              = 33 # number of channels -- filters
dim_branch_out_beta  = 200


dim_DeepO_in_beta   = dim_trunc_out_beta + dim_branch_out_beta
dim_DeepO_out_beta  = dimZ * 2

kip_res       = tf.constant([10, 10, 10])
grid_kip_beta = raveled3DDomainGrid( kip_res )

gicNetBeta = GICNet( grid_kip_beta, kip_res, dim_branch_in_y_beta, dim_branch_P_beta, dimX  )

gicNetBeta.P_kLayers   = [ tf.keras.layers.Dense( dim_branch_P_beta, input_dim=( dim_branch_in_y_beta ), activation ='swish'),
                        tf.keras.layers.Dense( dim_branch_P_beta, activation ='swish'),
                        tf.keras.layers.Dense( dim_branch_P_beta, activation ='swish'),
                        tf.keras.layers.Dense( dim_branch_P_beta, activation ='swish') ]
                        
gicNetBeta.kipLayers   = [ ]
ellXs = tf.linspace(0., 1., dim_branch_P_beta)
low  = 1e-3
high = 1.
ellVals = expHighLowFunc_01(low, high, ellXs)
for i in range(dim_branch_P_beta):

        gicNetBeta.kipLayers.append(KIPLayer(EQ(), ellVals[i]))
        print('kip layer {} ell = '.format(i), gicNetBeta.kipLayers[i].ell)


gicNetBeta.FFMlayer    = [ ]

gicNetBeta.preTrunclayers  = [ 
                        tf.keras.layers.Dense( dim_trunc_out_beta, input_dim=( dim_trunc_in_beta ), activation ='swish'), 
                        tf.keras.layers.Dense( dim_trunc_out_beta,                                  activation ='swish'), 
                        tf.keras.layers.Dense( dim_trunc_out_beta,                                  activation ='swish')
                        ]

swap_txy_layer = lambda x: tf.transpose(x, [0, 3, 1, 2, 4]) 
swap_xyt_layer = lambda x: tf.transpose(x, [0, 2, 3, 1, 4]) 

gicNetBeta.convLayers = [       # 0      1         2         3         4
                                #[batch, conv_1 x, conv_2 y, conv_3 t, channel]
                                #[batch, batch , conv_2, conv_3, channel]
                                #[batch, batch , batch, conv_1, channel]
                        # xyt
                        tf.keras.layers.Lambda(swap_txy_layer), #txy
                        tf.keras.layers.Conv2D( dv_beta, (2,2), strides=(1,1), padding='valid', activation='swish' ),
                        tf.keras.layers.Lambda(swap_xyt_layer), # xyt
                        tf.keras.layers.Conv1D( dv_beta, (2), strides=(1), padding='valid', activation='swish' ),

                        tf.keras.layers.Lambda(swap_txy_layer), #txy
                        tf.keras.layers.Conv2D( dv_beta, (2,2), strides=(1,1), padding='valid', activation='swish' ),
                        tf.keras.layers.Lambda(swap_xyt_layer), # xyt
                        tf.keras.layers.Conv1D( dv_beta, (2), strides=(1), padding='valid', activation='swish' ),

                        tf.keras.layers.Lambda(swap_txy_layer), #txy
                        tf.keras.layers.Conv2D( dv_beta, (2,2), strides=(1,1), padding='valid', activation='swish' ),
                        tf.keras.layers.Lambda(swap_xyt_layer), # xyt
                        tf.keras.layers.Conv1D( dv_beta, (2), strides=(1), padding='valid', activation='swish' ),

                        tf.keras.layers.Lambda(swap_txy_layer), #txy
                        tf.keras.layers.Conv2D( dv_beta, (2,2), strides=(1,1), padding='valid', activation='swish' ),
                        tf.keras.layers.Lambda(swap_xyt_layer), # xyt
                        tf.keras.layers.Conv1D( dv_beta, (2), strides=(1), padding='valid', activation='swish' ),


                        tf.keras.layers.Flatten(  ),
                        tf.keras.layers.Dense(1000),
                        tf.keras.layers.Dense(dim_branch_out_beta),
                        ]
                        
nn  = tf.keras.Sequential( gicNetBeta.convLayers  )
val = tf.random.uniform(shape=[1,kip_res[0], kip_res[1], kip_res[2], dim_branch_P_beta + dimX])
nn(val)
nn.summary()
nn, val = None, None

nhBeta       = 400
deep0_beta   = tf.keras.layers.Dense( nhBeta, input_dim=( dim_DeepO_in_beta ), activation=tf.keras.activations.swish)
deep1_beta   = tf.keras.layers.Dense( nhBeta, activation=tf.keras.activations.swish)
deep2_beta   = tf.keras.layers.Dense( nhBeta, activation=tf.keras.activations.swish)
deep3_beta   = tf.keras.layers.Dense( nhBeta, activation=tf.keras.activations.swish)
deep2_beta   = tf.keras.layers.Dense( nhBeta, activation=tf.keras.activations.swish)
deep3_beta   = tf.keras.layers.Dense( nhBeta, activation=tf.keras.activations.swish)
deepOut_beta = tf.keras.layers.Dense( dim_DeepO_out_beta )
gicNetBeta.DeepOlayers = [ deep0_beta, deep1_beta, deep2_beta, deep3_beta, deepOut_beta ]


betaNet    = NeuralN('beta', np.array([ 1, 1, dimZ]) )
betaNet.NN = gicNetBeta

betaNet.lower_alpha  = tf.constant(1e-4) 
betaNet.upper_w      = tf.constant(1.)

def betaMap(self, u, w, x):

        trunc_in  = tf.reshape(w, [-1,dimW])
        branch_in = tf.concat([x, u], 1)

        mapped = self.NN( branch_in, trunc_in )
        print('mapped.shape = ', mapped.shape)

        mean, logvar = mapped[:,:dimZ], mapped[:,dimZ:2*dimZ]

        tf.debugging.assert_all_finite(mean, 'mean not finite')
        tf.debugging.assert_all_finite(logvar, 'logvar not finite')

        logvar = self.dBoundwalpha_log( logvar )
        print('beta mean.shape = ', mean.shape)
        print('beta logvar.shape = ', logvar.shape)

        return mean, logvar

from types import MethodType
betaNet.call = MethodType(betaMap, betaNet)

mean_alpha, logvar_alpha = alphaNet(tf.constant(1., shape=[dimZ]), tf.constant(1., shape=[dimW]), xtvect)
print(mean_alpha.shape)

mean_beta, logvar_beta = betaNet( mean_alpha, tf.constant(1., shape=[dimW]), xtvect )
betaNet.compile( )
betaNet.summary( )
betaNet.NN.summary( )



# betaNet.save_weights(dirname + '/betaModel/betaModel-lfmDiag')
# betaNet.load_weights(dirname + '/betaModel/betaModel-lfmDiag')

# del betaNet


In [ ]:

def D_square(x, y, t):
    return tf.sin(pi * x) * tf.sin(pi * y) * t

power_n = 6
def base_u(x, y, t, w):
    return w * ( 1. - (2*x-1)**power_n ) * y * t

def D_p(x, y, t):
    return x + y - e_bound * 2.


alphaNet.epsilon_r = tf.Variable(0.01, trainable=False)
# epsilon_r = tf.Variable(1e-2, trainable=False)

epsiBij = tfp.bijectors.SoftClip(low=1e-5, high=1.)

def residualFunction( z, w, xyt ):
    
    x,y, t = xyt[:, 0:1], xyt[:, 1:2], xyt[:, 2:3]
    print('x.shape', x.shape)
    print('y.shape', y.shape)
    print('t.shape', t.shape)

    # z[0] = rho, z[1] = mu
    z = tf.reshape(z, [-1])
    w = tf.reshape(w, [ ])

    with tf.GradientTape(watch_accessed_variables=False, persistent=True) as t1:
        t1.watch( x )
        t1.watch( y )

        with tf.GradientTape(watch_accessed_variables=False, persistent=True) as t2:
            t2.watch( x )
            t2.watch( y )
            t2.watch( t )

            mean_alpha, logvar_alpha = alphaNet( z, w, tf.concat([x,y,t], 1) )

            mean_alpha_stack   = tf.reshape(mean_alpha,   [-1,1])
            logvar_alpha_stack = tf.reshape(logvar_alpha, [-1,1])

            uvp_untrans = tf.reshape(reparameterize(mean_alpha_stack, logvar_alpha_stack), [-1, dimU])

            u_untrans, v_untrans, p_untrans = uvp_untrans[:, 0:1], uvp_untrans[:, 1:2], uvp_untrans[:, 2:3]

            u = base_u(x, y, t, w) + D_square(x, y, t) * u_untrans
            v =                      D_square(x, y, t) * v_untrans
            p =                      D_p(x, y, t)      * p_untrans
            
        u_x = tf.reshape(t2.gradient(u, x), [-1,1])
        u_y = tf.reshape(t2.gradient(u, y), [-1,1])
        u_t = tf.reshape(t2.gradient(u, t), [-1,1])

        v_x = tf.reshape(t2.gradient(v, x), [-1,1])
        v_y = tf.reshape(t2.gradient(v, y), [-1,1])
        v_t = tf.reshape(t2.gradient(v, t), [-1,1])

        p_x = tf.reshape(t2.gradient(p, x), [-1,1])
        p_y = tf.reshape(t2.gradient(p, y), [-1,1])

    u_xx = tf.reshape(t1.gradient(u_x, x), [-1,1])
    u_yy = tf.reshape(t1.gradient(u_y, y), [-1,1])

    v_xx = tf.reshape(t1.gradient(v_x, x), [-1,1])
    v_yy = tf.reshape(t1.gradient(v_y, y), [-1,1])
    
    rho, mu = tf.squeeze(z[0]),  tf.squeeze(z[1])
    res_1 = rho * u_t + rho * ( u * u_x + v * u_y ) + p_x - mu * ( u_xx + u_yy )
    res_2 = rho * v_t + rho * ( u * v_x + v * v_y ) + p_y - mu * ( v_xx + v_yy )
    res_3 = u_x + v_y

    res   = tf.concat([res_1, res_2, res_3 * 10.], 0) 
    
    tf.debugging.assert_all_finite(res, 'res not finite')
    
    print("residualBody.shape", res.shape)
    print('u.shape', u.shape)
    print('mean_alpha.shape', mean_alpha.shape)
    print('logvar_alpha.shape', logvar_alpha.shape )
        
    logP_alpha_u_Z = log_normal_pdf(tf.reshape(uvp_untrans, [-1,1]), mean_alpha_stack, logvar_alpha_stack, raxis=0)

    return uvp_untrans, res, logP_alpha_u_Z


def ELBO( zw ):

    z,w    = zw[:dimZ], zw[dimZ:dimZ+dimW]
    xtrand = sampleDomain(nxSamples)

    print("ELBO xrand.shape", xtrand.shape)
    print('ELBO z.shape', z.shape)
    print('ELBO w.shape', w.shape)

    uvp_untrans, residual, logP_alpha_u_Z = residualFunction( z, w, xtrand )

    logVarEpsVect  = tf.repeat(2.*tf.math.log(alphaNet.epsilon_r), residual.shape[0] )[:, None]

    logP_r   = log_normal_pdf( residual, tf.constant(0., shape=residual.shape), logVarEpsVect, raxis=0 )
    
    mean_beta, logvar_beta = betaNet( tf.reshape(uvp_untrans, [-1,dimU]), w, xtrand )

    print('mean_beta.shape', mean_beta.shape)
    print('mean_beta.shape', logvar_beta.shape)

    logP_beta_Z_u          = log_normal_pdf( tf.reshape(z, [1,-1]), mean_beta, logvar_beta, raxis=1 ) #this is correct dimension

    print('logP_alpha_u_Z.shape', logP_alpha_u_Z.shape)
    print('logP_beta_Z_u.shape', logP_beta_Z_u.shape)
    print('logP_r.shape', logP_r.shape)

    tf.debugging.assert_all_finite(logP_r, 'logP_r not finite')
    tf.debugging.assert_all_finite(logP_beta_Z_u, 'logP_beta_Z_u not finite')
    tf.debugging.assert_all_finite(logP_alpha_u_Z, 'logP_alpha_u_Z not finite')

    elbo = tf.reduce_mean( logP_r + logP_beta_Z_u -  logP_alpha_u_Z )

    return elbo

zs =  zDist.sample( )
ws =  wDist.sample( )

ELBO( tf.concat([zs, ws], 0) )

In [ ]:


trainableVars   = [ alphaNet.trainable_variables, betaNet.trainable_variables]

@tf.function
def train_step(optimizer):

    zs =  zDist.sample( batchSize )
    ws =  wDist.sample( batchSize )

    print('zs.shape', zs.shape)
    print('ws.shape', ws.shape)

    zws = tf.concat([zs, ws], 1)

    print('zwxs.shape', zws.shape)

    with tf.GradientTape(persistent=True) as tape:

        losses = tf.vectorized_map(ELBO, zws, fallback_to_while_loop=False)
    
        loss   = - tf.reduce_mean(losses)
        tf.debugging.assert_all_finite(loss, 'loss not finite, {}'.format(losses))
        
    for vars in trainableVars:
        gradients = tape.gradient(loss, vars)
        optimizer.apply_gradients(zip(gradients, vars))

    return loss

batchSize = 5

num_iterations   = tf.constant(1_000_000)


intervalELBOsave = 100

lr = tf.keras.optimizers.schedules.ExponentialDecay(
    1e-3, int(num_iterations/batchSize/5), 0.5, staircase=True, name=None
)
optimizer = tf.keras.optimizers.Adam( lr )


elboAll     = []
epsAll      = []
startTrainingPDDLVM = time.time()
for iteration in tf.range(0, int(num_iterations/batchSize) ):

    start_time = time.time()
    loss = train_step(optimizer)

    end_time = time.time()

    if iteration % intervalELBOsave == 0:
        epsAll.append( alphaNet.epsilon_r.numpy() )
        elboAll.append( -loss ) 
        print('Epoch: {}, Test set ELBO: {:.2f}, epsilon_r {:.4f}, time elapse for current epoch: {:.4f}'
        .format(iteration, -loss, alphaNet.epsilon_r.numpy(), end_time - start_time))

endTrainingPDDLVM = time.time()
totalTimeTrainingPDLVM = (endTrainingPDDLVM - startTrainingPDDLVM)/60.

print('totalTimeTrainingPDLVM = {:.2f}m'.format( totalTimeTrainingPDLVM ) )




In [ ]:
saveWeights       = False
overPowerSaveConf = False

loadWeights = True

model_name = '0_8-1--0_1-1--1-1--phylvmdiag/'

if saveWeights == True:
    saveWeightsConf = 'n'
    if overPowerSaveConf == False:
        saveWeightsConf = input('you want to save/overwrite weights? Y/n')
    if saveWeightsConf == 'Y' or overPowerSaveConf == True:
        alphaNet.save_weights(dirname+model_name+'alphaModel/checkpoints/my_checkpoint')
        betaNet.save_weights(dirname+model_name+'betaNet/checkpoints/my_checkpoint')
        print('weights have been saved')

if loadWeights == True:
    loadWeightsConf = input('you want to load weights? Y/n')
    if loadWeightsConf == 'Y':
        alphaNet.load_weights(dirname+model_name+'alphaModel/checkpoints/my_checkpoint')
        betaNet.load_weights(dirname+model_name+'betaNet/checkpoints/my_checkpoint')
        print('weights have been loaded')


In [8]:
# os.popen('cp ./phy-lvm-diag.ipynb {}phy-lvm-diag.ipynb'.format(dirname))

savefile_python = True
if savefile_python == True:
    filename = 'phy-lvm-diag-smallColLongTrain-1_2'
    os.popen('ipynb-py-convert ./{0}.ipynb {1}{2}{0}.py'.format(filename, dirname, model_name))

In [ ]:
ebloAll = tf.stack(elboAll)
LOSS    = np.array(ebloAll)
plt.plot(range(LOSS.shape[0]), LOSS )
plt.show()
print(np.isnan(LOSS).mean())
plt.plot(range(LOSS.shape[0]), np.log(-LOSS) )
plt.show()
stack_epsAll = tf.stack(epsAll)
plt.plot(range( len(epsAll)), stack_epsAll )
plt.show()

np.savetxt(dirname  + 'ebloAll.dat' , ebloAll )
np.savetxt(dirname  + 'epsAll.dat'  , stack_epsAll )
print('totalTimeTrainingPDLVM = {:.2f}m'.format( totalTimeTrainingPDLVM ) )


In [ ]:
z = [1., 1.]
w = [1.]
print('plotting')
resoTEST = [100, 100, 10]
pt = 9
X, Y, T = np.meshgrid( tf.linspace(0., 1., resoTEST[0]), tf.linspace(0., 1., resoTEST[1]), tf.linspace(0., 1., resoTEST[2]), indexing='xy')

xr, yr, tr = tf.reshape(X, [-1,1]), tf.reshape(Y, [-1,1]), tf.reshape(T, [-1,1])

mean_alpha, logvar_alpha = alphaNet( z, w, tf.concat([xr, yr, tr], 1) )

mean_alpha_stack   = tf.reshape(mean_alpha,   [-1,1])
logvar_alpha_stack = tf.reshape(logvar_alpha, [-1,1])

uvp_untrans = mean_alpha


u_untrans, v_untrans, p_untrans          = uvp_untrans[:, 0:1], uvp_untrans[:, 1:2], uvp_untrans[:, 2:3]
u_untrans_lv, v_untrans_lv, p_untrans_lv = logvar_alpha[:, 0:1], logvar_alpha[:, 1:2], logvar_alpha[:, 2:3]

u_vec = base_u(xr, yr, tr, w) + D_square(xr, yr, tr) * u_untrans
v_vec =                         D_square(xr, yr, tr) * v_untrans
p_vec =                         D_p(xr, yr, tr)      * p_untrans

P_vec = tf.reshape(p_vec, resoTEST)
U_vec = tf.reshape(u_vec, resoTEST)
V_vec = tf.reshape(v_vec, resoTEST)


u_vec_std =  tf.sqrt( D_square(xr, yr, tr)**2. * tf.exp( u_untrans_lv ) )
v_vec_std =  tf.sqrt( D_square(xr, yr, tr)**2. * tf.exp( v_untrans_lv ) )
p_vec_std =  tf.sqrt( D_p(xr, yr, tr)     **2. * tf.exp( p_untrans_lv ) )

P_vec_std = tf.reshape(p_vec_std, resoTEST)
U_vec_std = tf.reshape(u_vec_std, resoTEST)
V_vec_std = tf.reshape(v_vec_std, resoTEST)


fig, ax = plt.subplots()

########### plot slns ###########
mag_vec = np.array( tf.linalg.norm(tf.concat([U_vec[...,pt:pt+1], V_vec[...,pt:pt+1]],-1), axis=-1))

strm  = ax.streamplot(X[...,pt], Y[...,pt], U_vec[...,pt], V_vec[...,pt], color=mag_vec, density=8,cmap = plt.cm.seismic,
                        broken_streamlines=False, linewidth=0.1, arrowsize=0.)  
# strm  = ax.quiver(X, Y,U_vec, V_vec)
# ax.contourf(X,Y, P_vec, 50)
fig.colorbar(strm.lines, ax=ax)
ax.set(xlabel=r'$x$', ylabel=r'$y$')
ax.set_xlim(0., 1.)
ax.set_ylim(0., 1.)
plt.tight_layout()
plt.savefig(dirname + 'strlines.png', dpi=300)
plt.show()

plt.contourf(X[...,pt],Y[...,pt], P_vec[...,pt], 50)
plt.colorbar()
plt.xlim(0., 1.)
plt.ylim(0., 1.)
plt.ylabel(r'$t$')
plt.xlabel(r'$x$')
plt.tight_layout()
plt.savefig(dirname + 'P.png', dpi=300)
plt.show()
plt.close()

plt.contourf(X[...,pt],Y[...,pt], U_vec[...,pt], 50)
plt.colorbar()
plt.xlim(0., 1.)
plt.ylim(0., 1.)
plt.ylabel(r'$t$')
plt.xlabel(r'$x$')
plt.tight_layout()
plt.savefig(dirname + 'U.png', dpi=300)
plt.show()
plt.close()


plt.contourf(X[...,pt],Y[...,pt], V_vec[...,pt], 50)
plt.colorbar()
plt.xlim(0., 1.)
plt.ylim(0., 1.)
plt.ylabel(r'$t$')
plt.xlabel(r'$x$')
plt.tight_layout()
plt.savefig(dirname + 'V.png', dpi=300)
plt.show()
plt.close()

########### plot stddev ###########
fig, ax = plt.subplots()

mag_vec_std = np.array( tf.linalg.norm(tf.concat([U_vec_std[...,pt:pt+1], V_vec_std[...,pt:pt+1]],-1), axis=-1))

strm  = ax.streamplot(X[...,pt], Y[...,pt], U_vec_std[...,pt], V_vec_std[...,pt], color=mag_vec, density=8,cmap = plt.cm.seismic,
                        broken_streamlines=False, linewidth=0.1, arrowsize=0.)  
# strm  = ax.quiver(X, Y,U_vec, V_vec)
# ax.contourf(X,Y, P_vec, 50)
# fig.colorbar(strm.lines, ax=ax)
# ax.set(xlabel=r'$x$', ylabel=r'$y$')
# ax.set_xlim(0., 1.)
# ax.set_ylim(0., 1.)
# plt.ylabel(r'$t$')
# plt.xlabel(r'$x$')
# plt.tight_layout()
# plt.savefig(dirname + 'strlines_std.png', dpi=300)
# plt.show()

plt.contourf(X[...,pt],Y[...,pt], P_vec_std[...,pt], 50)
plt.colorbar()
plt.xlim(0., 1.)
plt.ylim(0., 1.)
plt.ylabel(r'$t$')
plt.xlabel(r'$x$')
plt.tight_layout()
plt.savefig(dirname + 'Pressure_std.png', dpi=300)
plt.show()
plt.close()

plt.contourf(X[...,pt],Y[...,pt], U_vec_std[...,pt], 50)
plt.colorbar()
plt.xlim(0., 1.)
plt.ylim(0., 1.)
plt.ylabel(r'$t$')
plt.xlabel(r'$x$')
plt.tight_layout()
plt.savefig(dirname + 'U_std.png', dpi=300)
plt.show()
plt.close()


plt.contourf(X[...,pt],Y[...,pt], V_vec_std[...,pt], 50)
plt.colorbar()
plt.xlim(0., 1.)
plt.ylim(0., 1.)
plt.ylabel(r'$t$')
plt.xlabel(r'$x$')
plt.tight_layout()
plt.savefig(dirname + 'V_std.png', dpi=300)
plt.show()
plt.close()

mean_beta, logvar_beta = betaNet( tf.reshape(uvp_untrans, [-1,dimU]), w[0], tf.concat([xr, yr, tr], 1) ) 

print('mean_beta = ', mean_beta)
print('std_beta = ', tf.exp(logvar_beta * 0.5) )


In [ ]:
dirnameGT = '../../../fenics/NSLid/data-groundTruth-nslid_100_0_8-1--0_1-1--1-1'

# saved i, T, X, Y
# want  i, X, Y, T
P_vec_all = tf.constant( np.load(dirnameGT+'/'+'SolutionsPGT.npy'), dtype=tf.float32)
U_vec_all = tf.constant( np.load(dirnameGT+'/'+'SolutionsUGT.npy'), dtype=tf.float32)
V_vec_all = tf.constant( np.load(dirnameGT+'/'+'SolutionsVGT.npy'), dtype=tf.float32)

pad       = 1
padt      = 1
P_vec_all = tf.transpose(P_vec_all, [0, 2, 3, 1])[:, pad:-pad, pad:-pad, padt:]
U_vec_all = tf.transpose(U_vec_all, [0, 2, 3, 1])[:, pad:-pad, pad:-pad, padt:]
V_vec_all = tf.transpose(V_vec_all, [0, 2, 3, 1])[:, pad:-pad, pad:-pad, padt:]

allZ      = tf.constant( np.loadtxt(dirnameGT+'/'+'allZ.dat' )    , dtype=tf.float32)
allW      = tf.constant( np.loadtxt(dirnameGT+'/'+ 'allW.dat')    , dtype=tf.float32)
fenics_XY = tf.constant( np.load(dirnameGT+'/'+'FenicsMeshXY.npy'), dtype=tf.float32)[:, pad:-pad, pad:-pad]
fenics_T  = tf.constant( np.load(dirnameGT+'/'+'FenicsMeshT.npy') , dtype=tf.float32)[padt:]


print('P_vec_all.shape', P_vec_all.shape)
print('U_vec_all.shape', U_vec_all.shape)
print('V_vec_all.shape', V_vec_all.shape)

print('allZ     .shape', allZ     .shape)
print('allW     .shape', allW     .shape)
print('fenics_XY.shape', fenics_XY.shape)
print('fenics_T .shape', fenics_T .shape)

nx = fenics_XY.shape[1]
nt = fenics_XY.shape[2]

saveDataList = True

if saveDataList:

     allresAbsMean = []

     allSampleZs   = []
     allMeanZs     = []
     allStdZs      = []

     allUSamples = []
     allUMeans   = []
     allUStd     = []
        
     allVSamples = []
     allVMeans   = []
     allVStd     = []

     allZNSE       = []
     allUNSE       = []
     allVNSE       = []


     Pin2SigK = []
     Pin2SigU = []
     Pin2SigV = []

# XYT 3, 50,50,20
# XY  2, 50, 50
# T   20
fenics_XYtile = tf.tile(fenics_XY[..., None], [1,1,1,fenics_T.shape[0]])
fenics_Ttile  = tf.tile(fenics_T[None, None, None, :], [1, fenics_XY.shape[1], fenics_XY.shape[2], 1])
fenics_XYT    = tf.concat([fenics_XYtile, fenics_Ttile], 0)
print('fenicsXYT.shape', fenics_XYT.shape)

pt    = fenics_XYT.shape[3]-1
print('pt = ', pt)
plt.contourf(fenics_XYT[0,...,pt], fenics_XYT[1,...,pt], U_vec_all[0,...,pt], 50)
plt.colorbar( )
plt.xlim(0., 1.)
plt.ylim(0., 1.)
plt.ylabel(r'$y$')
plt.xlabel(r'$x$')
plt.tight_layout()
plt.show()
plt.close()

xr, yr, tr   = tf.reshape(fenics_XYT[0], [-1,1]), tf.reshape(fenics_XYT[1], [-1,1]), tf.reshape(fenics_XYT[2], [-1,1])
xyttest_rav  = tf.concat([xr, yr, tr], 1)
tf_residualFunction = residualFunction #
tf_alphaNet         = alphaNet         #
tf_betaNet          = betaNet          #


PLOT        = False
savePlots   = False
BenchParams = False
writeToFile = False

for i in range(U_vec_all.shape[0]):


     zTrue = tf.reshape( allZ[i], [-1, 1] )
     wTrue = tf.reshape( allW[i], [ ] )
     uTrue = tf.reshape( U_vec_all[i], [-1, 1] )
     vTrue = tf.reshape( V_vec_all[i], [-1, 1] )
     pTrue = tf.reshape( P_vec_all[i], [-1, 1] )

    
     if BenchParams:
          zTrue = tf.constant([1., 0.1])
          wTrue = tf.constant([1.])

     print('zTrue = ', zTrue)
     print('wTrue = ', wTrue)
     
     mean_alpha, logvar_alpha = tf_alphaNet( zTrue, wTrue, xyttest_rav )
     mean_alpha_stack   = tf.reshape(mean_alpha,   [-1,1])
     logvar_alpha_stack = tf.reshape(logvar_alpha, [-1,1])
 
#      uvp_untrans = tf.reshape(reparameterize(mean_alpha_stack, logvar_alpha_stack), [-1, dimU])
     uvp_untrans = mean_alpha
     u_untrans, v_untrans, p_untrans = uvp_untrans[:, 0:1], uvp_untrans[:, 1:2], uvp_untrans[:, 2:3]

     u_vec = base_u(xr, yr, tr, wTrue) + D_square(xr, yr, tr) * u_untrans
     v_vec =                             D_square(xr, yr, tr) * v_untrans
     p_vec =                             D_p(xr, yr, tr)      * p_untrans

     if PLOT == True:
          U_vec = tf.reshape(u_vec, fenics_XYT[0].shape)
          # plt.tricontourf(xr[:,0], yr[:,0], u_vec[:,0], 50)
          # plt.tricontourf(xr[:,0], yr[:,0], uTrue[:,0], 50)
          plt.contourf(fenics_XYT[0,...,pt],fenics_XYT[1,...,pt], U_vec[...,pt], 50)
          plt.colorbar()
          plt.xlim(0., 1.)
          plt.ylim(0., 1.)
          plt.ylabel(r'$t$')
          plt.xlabel(r'$x$')
          plt.tight_layout()
          plt.show()
          plt.close()

          UTrue = tf.reshape(uTrue, fenics_XYT[0].shape)
          # plt.tricontourf(xr[:,0], yr[:,0], u_vec[:,0], 50)
          # plt.tricontourf(xr[:,0], yr[:,0], uTrue[:,0], 50)
          plt.contourf(fenics_XYT[0,...,pt],fenics_XYT[1,...,pt], UTrue[...,pt], 50)
          plt.colorbar()
          plt.xlim(0., 1.)
          plt.ylim(0., 1.)
          plt.ylabel(r'$t$')
          plt.xlabel(r'$x$')
          plt.tight_layout()
          plt.show()
          plt.close()

          V_vec = tf.reshape(v_vec, fenics_XYT[0].shape)
          # plt.tricontourf(xr[:,0], yr[:,0], u_vec[:,0], 50)
          # plt.tricontourf(xr[:,0], yr[:,0], uTrue[:,0], 50)
          plt.contourf(fenics_XYT[0,...,pt],fenics_XYT[1,...,pt], V_vec[...,pt], 50)
          plt.colorbar()
          plt.xlim(0., 1.)
          plt.ylim(0., 1.)
          plt.ylabel(r'$t$')
          plt.xlabel(r'$x$')
          plt.tight_layout()
          plt.show()
          plt.close()

          VTrue = tf.reshape(vTrue, fenics_XYT[0].shape)
          # plt.tricontourf(xr[:,0], yr[:,0], u_vec[:,0], 50)
          # plt.tricontourf(xr[:,0], yr[:,0], uTrue[:,0], 50)
          plt.contourf(fenics_XYT[0,...,pt],fenics_XYT[1,...,pt], VTrue[...,pt], 50)
          plt.colorbar()
          plt.xlim(0., 1.)
          plt.ylim(0., 1.)
          plt.ylabel(r'$t$')
          plt.xlabel(r'$x$')
          plt.tight_layout()
          plt.show()
          plt.close()

     u_untrans_lv, v_untrans_lv, p_untrans_lv = logvar_alpha[:, 0:1], logvar_alpha[:, 1:2], logvar_alpha[:, 2:3]

     u_vec_std = tf.sqrt( D_square(xr, yr, tr) ** 2. * tf.exp( u_untrans_lv ) )
     v_vec_std = tf.sqrt( D_square(xr, yr, tr) ** 2. * tf.exp( v_untrans_lv ) )
     p_vec_std = tf.sqrt( D_p(xr, yr, tr)      ** 2. * tf.exp( p_untrans_lv ) )
     
     p_true_unstranf        = (pTrue ) / ( D_p(xr, yr, tr) )
     u_true_unstranf        = (uTrue - base_u(xr, yr, tr, wTrue)) / ( D_square(xr, yr, tr) )
     v_true_unstranf        = (vTrue ) / ( D_square(xr, yr, tr) )
     uvp_true_untransf      = tf.concat([u_true_unstranf, v_true_unstranf, p_true_unstranf], 1)
     tf.debugging.assert_all_finite(uvp_true_untransf, 'uvp_untrsanf not finite')
     mean_beta, logvar_beta = tf_betaNet( uvp_true_untransf, wTrue,  xyttest_rav )

     mean_beta, logvar_beta = tf.squeeze( mean_beta), tf.squeeze( logvar_beta )
     sampleZ  = reparameterize( mean_beta, logvar_beta )
     std_beta = tf.exp( 0.5 * logvar_beta)

     if saveDataList:

          allSampleZs.append(tf.squeeze(sampleZ))
          allMeanZs.append(tf.squeeze(mean_beta))
          allStdZs.append(std_beta)

          allUSamples.append(tf.squeeze(u_vec))
          allVSamples.append(tf.squeeze(v_vec))
        
          allUStd.append(tf.squeeze(u_vec_std))
          allVStd.append(tf.squeeze(v_vec_std))
          
          allZNSE.append( ( tf.linalg.norm((tf.squeeze(mean_beta) - tf.squeeze(zTrue)) ) / tf.linalg.norm(tf.squeeze(zTrue)) )**2. )

          allUNSE.append( ( tf.linalg.norm((tf.squeeze(u_vec) - tf.squeeze(uTrue)) ) / tf.linalg.norm(tf.squeeze(uTrue)) )**2. )
          allVNSE.append( ( tf.linalg.norm((tf.squeeze(v_vec) - tf.squeeze(vTrue)) ) / tf.linalg.norm(tf.squeeze(vTrue)) )**2. )

          Pin2SigK.append( tf.reduce_sum(tf.where(tf.math.logical_and( 
               tf.squeeze(zTrue) >  tf.squeeze(mean_beta-2*std_beta), tf.squeeze(zTrue) <  tf.squeeze(mean_beta+2*std_beta)
               ), 1, 0)) /  tf.squeeze(zTrue).shape[0] )

          Pin2SigU.append( tf.reduce_sum(tf.where(tf.math.logical_and(
               tf.squeeze(uTrue) > tf.squeeze(u_vec-2*u_vec_std), tf.squeeze(uTrue) < tf.squeeze(u_vec+2*u_vec_std)
               ), 1, 0)) / tf.squeeze(uTrue).shape[0]  )
        
          Pin2SigV.append( tf.reduce_sum(tf.where(tf.math.logical_and(
               tf.squeeze(vTrue) > tf.squeeze(v_vec-2*v_vec_std), tf.squeeze(vTrue) < tf.squeeze(v_vec+2*v_vec_std)
               ), 1, 0)) / tf.squeeze(vTrue).shape[0]  )


print('mean allZNSE = ', tf.reduce_mean( tf.stack(allZNSE) ) )
print('mean allUNSE = ', tf.reduce_mean( tf.stack(allUNSE) ) )
print('mean allVNSE = ', tf.reduce_mean( tf.stack(allVNSE) ) )

print('sttdev allZNSE = ', tf.math.reduce_std( tf.stack(allZNSE) ) )
print('sttdev allUNSE = ', tf.math.reduce_std( tf.stack(allUNSE) ) )
print('sttdev allVNSE = ', tf.math.reduce_std( tf.stack(allVNSE) ) )


# print('median allZNSE = ', tfp.stats.percentile( allZNSE, 50 ) )

print('Pin2SigK = ', tf.reduce_mean(tf.stack(Pin2SigK)))
print('Pin2SigU = ', tf.reduce_mean(tf.stack(Pin2SigU)))
print('Pin2SigV = ', tf.reduce_mean(tf.stack(Pin2SigV)))

# print('allUStd = ', tf.reduce_mean(tf.stack(allUStd)))

if writeToFile:
     with  open(dirname+"allInfoRun.txt", "w") as f:
          f.write('mean allZNSE = {}\n'.format(tf.reduce_mean( tf.stack(allZNSE) )) )
          f.write('mean allUNSE = {}\n'.format(tf.reduce_mean( tf.stack(allUNSE) )) )
          f.write('mean allVNSE = {}\n'.format(tf.reduce_mean( tf.stack(allVNSE) )) )
          f.write('sttdev allZNSE = {}\n'.format(tf.math.reduce_std( tf.stack(allZNSE) )) )
          f.write('sttdev allUNSE = {}\n'.format(tf.math.reduce_std( tf.stack(allUNSE) )) )
          f.write('sttdev allVNSE = {}\n'.format(tf.math.reduce_std( tf.stack(allVNSE) )) )
          f.write('Pin2SigK = {}\n'.format(tf.reduce_mean(tf.stack(Pin2SigK))) )
          f.write('Pin2SigU = {}\n'.format(tf.reduce_mean(tf.stack(Pin2SigU))) )
          f.write('Pin2SigV = {}\n'.format(tf.reduce_mean(tf.stack(Pin2SigV))) )
          f.write('totalTimeTrainingPDLVM = {:.2f}m\n'.format( totalTimeTrainingPDLVM ))
     f.close()

print('totalTimeTrainingPDLVM = {:.2f}m'.format( totalTimeTrainingPDLVM ) )



raise

In [ ]:
saveData = True
if saveData:

    allSampleZs_np = tf.stack(allSampleZs)
    allMeanZs_np   = tf.stack(allMeanZs)
    allStdZs_np    = tf.stack(allStdZs)

    allUSamples_np  = tf.stack(allUSamples)
    allUMeans_np    = tf.stack(allUMeans)
    allUStd_np      = tf.stack(allUStd)

    prefix = 'randgrid{}_{}kIter'.format(nxSamples, int(num_iterations/1000))

    np.savetxt(dirname + prefix + '_allRes_np.dat'   , tf.stack(tf.squeeze(allresAbsMean)) )


    np.savetxt(dirname + prefix + '_allSampleZs_np.dat', allSampleZs_np)
    np.savetxt(dirname + prefix + '_allMeanZs_np.dat'  , allMeanZs_np)
    np.savetxt(dirname + prefix + '_allStdZs_np.dat'  , allMeanZs_np)


    np.savetxt(dirname + prefix + '_allUSamples_np.dat' , allUSamples_np)
    np.savetxt(dirname + prefix + '_allUMeans_np.dat'   , allUMeans_np)
    np.savetxt(dirname + prefix + '_allUStd_np.dat'     , allUStd_np)
    
    np.savetxt(dirname + prefix + '_allZNSE_np.dat'   , tf.stack( allZNSE ) )
    np.savetxt(dirname + prefix + '_allUNSE_np.dat'   , tf.stack( allUNSE ) )

In [ ]:
for layer in betaNet.NN.kipLayers:
    print(layer.ell)